Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

C:\Users\K\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [14]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    #shape = hidden.get_shape().as_list()
   # print("*****",shape)  # 14*14*16
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    #print("####",shape)   7*7*16
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [11]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    print("the index is :",offset)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
the index is : 0
Minibatch loss at step 0: 4.290326
Minibatch accuracy: 6.2%
Validation accuracy: 11.0%
the index is : 16
the index is : 32
the index is : 48
the index is : 64
the index is : 80
the index is : 96
the index is : 112
the index is : 128
the index is : 144
the index is : 160
the index is : 176
the index is : 192
the index is : 208
the index is : 224
the index is : 240
the index is : 256
the index is : 272
the index is : 288
the index is : 304
the index is : 320
the index is : 336
the index is : 352
the index is : 368
the index is : 384
the index is : 400
the index is : 416
the index is : 432
the index is : 448
the index is : 464
the index is : 480
the index is : 496
the index is : 512
the index is : 528
the index is : 544
the index is : 560
the index is : 576
the index is : 592
the index is : 608
the index is : 624
the index is : 640
the index is : 656
the index is : 672
the index is : 688
the index is : 704
the index is : 720
the index is : 736
the index is : 7

the index is : 6080
the index is : 6096
the index is : 6112
the index is : 6128
the index is : 6144
the index is : 6160
the index is : 6176
the index is : 6192
the index is : 6208
the index is : 6224
the index is : 6240
the index is : 6256
the index is : 6272
the index is : 6288
the index is : 6304
the index is : 6320
the index is : 6336
the index is : 6352
the index is : 6368
the index is : 6384
the index is : 6400
Minibatch loss at step 400: 0.190827
Minibatch accuracy: 100.0%
Validation accuracy: 82.0%
the index is : 6416
the index is : 6432
the index is : 6448
the index is : 6464
the index is : 6480
the index is : 6496
the index is : 6512
the index is : 6528
the index is : 6544
the index is : 6560
the index is : 6576
the index is : 6592
the index is : 6608
the index is : 6624
the index is : 6640
the index is : 6656
the index is : 6672
the index is : 6688
the index is : 6704
the index is : 6720
the index is : 6736
the index is : 6752
the index is : 6768
the index is : 6784
the index

Validation accuracy: 83.5%
the index is : 12016
the index is : 12032
the index is : 12048
the index is : 12064
the index is : 12080
the index is : 12096
the index is : 12112
the index is : 12128
the index is : 12144
the index is : 12160
the index is : 12176
the index is : 12192
the index is : 12208
the index is : 12224
the index is : 12240
the index is : 12256
the index is : 12272
the index is : 12288
the index is : 12304
the index is : 12320
the index is : 12336
the index is : 12352
the index is : 12368
the index is : 12384
the index is : 12400
the index is : 12416
the index is : 12432
the index is : 12448
the index is : 12464
the index is : 12480
the index is : 12496
the index is : 12512
the index is : 12528
the index is : 12544
the index is : 12560
the index is : 12576
the index is : 12592
the index is : 12608
the index is : 12624
the index is : 12640
the index is : 12656
the index is : 12672
the index is : 12688
the index is : 12704
the index is : 12720
the index is : 12736
the ind

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [20]:
tf.reset_default_graph()
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 *  num_channels, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    pool = tf.nn.max_pool(data, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    hidden = tf.nn.relu(pool)
    shape = hidden.get_shape().as_list()
    print("*****",shape)  # 14*14*16
    pool = tf.nn.max_pool(pool, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    hidden = tf.nn.relu(pool)
    shape = hidden.get_shape().as_list()
    print("####",shape)  # 7*7*16
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

***** [16, 14, 14, 1]
#### [16, 7, 7, 1]
***** [10000, 14, 14, 1]
#### [10000, 7, 7, 1]
***** [10000, 14, 14, 1]
#### [10000, 7, 7, 1]


In [21]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #print("the index is :",offset)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.582308
Minibatch accuracy: 6.2%
Validation accuracy: 8.4%
Minibatch loss at step 50: 2.212067
Minibatch accuracy: 25.0%
Validation accuracy: 10.3%
Minibatch loss at step 100: 2.100518
Minibatch accuracy: 37.5%
Validation accuracy: 26.4%
Minibatch loss at step 150: 2.012785
Minibatch accuracy: 37.5%
Validation accuracy: 37.6%
Minibatch loss at step 200: 1.969894
Minibatch accuracy: 25.0%
Validation accuracy: 45.9%
Minibatch loss at step 250: 1.435222
Minibatch accuracy: 62.5%
Validation accuracy: 43.0%
Minibatch loss at step 300: 1.432277
Minibatch accuracy: 68.8%
Validation accuracy: 60.0%
Minibatch loss at step 350: 1.340147
Minibatch accuracy: 87.5%
Validation accuracy: 66.9%
Minibatch loss at step 400: 1.161930
Minibatch accuracy: 62.5%
Validation accuracy: 67.2%
Minibatch loss at step 450: 1.293348
Minibatch accuracy: 75.0%
Validation accuracy: 69.5%
Minibatch loss at step 500: 1.222820
Minibatch accuracy: 68.8%
Validation accuracy: 71.0%
Min

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

# 加入drop out

In [33]:
tf.reset_default_graph()
batch_size = 16
patch_size = 5
dropout_rate = 0.3  # == 1 - keep_prob
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  training = tf.placeholder_with_default(False, shape=(), name='training')
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 *  num_channels, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    pool = tf.nn.max_pool(data, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    hidden1 = tf.nn.relu(pool)
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training) # training 时才drop_out
    shape = hidden1_drop.get_shape().as_list()
    print("*****",shape)  # 14*14*16
    pool = tf.nn.max_pool(pool, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    hidden2 = tf.nn.relu(pool)
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training) # training 时才drop_out
    shape = hidden2_drop.get_shape().as_list()
    print("####",shape)  # 7*7*16
    reshape = tf.reshape( hidden2_drop, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden3, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

***** [16, 14, 14, 1]
#### [16, 7, 7, 1]
***** [10000, 14, 14, 1]
#### [10000, 7, 7, 1]
***** [10000, 14, 14, 1]
#### [10000, 7, 7, 1]


In [34]:
num_steps = 1000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #print("the index is :",offset)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,training: True}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.877772
Minibatch accuracy: 0.0%
Validation accuracy: 11.2%
Minibatch loss at step 50: 2.154980
Minibatch accuracy: 25.0%
Validation accuracy: 11.2%
Minibatch loss at step 100: 2.069037
Minibatch accuracy: 31.2%
Validation accuracy: 30.5%
Minibatch loss at step 150: 1.915334
Minibatch accuracy: 37.5%
Validation accuracy: 48.5%
Minibatch loss at step 200: 1.880721
Minibatch accuracy: 31.2%
Validation accuracy: 44.9%
Minibatch loss at step 250: 1.484746
Minibatch accuracy: 56.2%
Validation accuracy: 45.1%
Minibatch loss at step 300: 1.547102
Minibatch accuracy: 62.5%
Validation accuracy: 57.5%
Minibatch loss at step 350: 1.408836
Minibatch accuracy: 50.0%
Validation accuracy: 67.2%
Minibatch loss at step 400: 1.443589
Minibatch accuracy: 37.5%
Validation accuracy: 66.2%
Minibatch loss at step 450: 1.344938
Minibatch accuracy: 56.2%
Validation accuracy: 67.5%
Minibatch loss at step 500: 1.470896
Minibatch accuracy: 50.0%
Validation accuracy: 70.3%
Mi

In [35]:
# 不知道为啥效果差了

# LR decay


In [ ]:
with tf.name_scope("train"):       # not shown in the book
    initial_learning_rate = 0.1
    decay_steps = 10000
    decay_rate = 1/10
    global_step = tf.Variable(0, trainable=False, name="global_step")
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step,
                                               decay_steps, decay_rate)
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss, global_step=global_step)

In [45]:
tf.reset_default_graph()
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 *  num_channels, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    pool = tf.nn.max_pool(data, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    hidden = tf.nn.relu(pool)
    shape = hidden.get_shape().as_list()
    print("*****",shape)  # 14*14*16
    pool = tf.nn.max_pool(pool, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    hidden = tf.nn.relu(pool)
    shape = hidden.get_shape().as_list()
    print("####",shape)  # 7*7*16
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
        
  initial_learning_rate = 0.1
  decay_steps = 100
  decay_rate = 1/10
  global_step = tf.Variable(0, trainable=False, name="global_step")
  learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step,
                                           decay_steps, decay_rate)

  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
#training_op = optimizer.minimize(loss, global_step=global_step)


  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

***** [16, 14, 14, 1]
#### [16, 7, 7, 1]
***** [10000, 14, 14, 1]
#### [10000, 7, 7, 1]
***** [10000, 14, 14, 1]
#### [10000, 7, 7, 1]


In [47]:
num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #print("the index is :",offset)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels)) 
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.422063
Minibatch accuracy: 0.0%
Validation accuracy: 13.6%
Minibatch loss at step 50: 2.100040
Minibatch accuracy: 31.2%
Validation accuracy: 20.8%
Minibatch loss at step 100: 1.747661
Minibatch accuracy: 43.8%
Validation accuracy: 39.4%
Minibatch loss at step 150: 1.622249
Minibatch accuracy: 62.5%
Validation accuracy: 54.3%
Minibatch loss at step 200: 1.499662
Minibatch accuracy: 56.2%
Validation accuracy: 61.3%
Minibatch loss at step 250: 0.800409
Minibatch accuracy: 87.5%
Validation accuracy: 64.7%
Minibatch loss at step 300: 1.196800
Minibatch accuracy: 68.8%
Validation accuracy: 71.0%
Minibatch loss at step 350: 0.916491
Minibatch accuracy: 68.8%
Validation accuracy: 72.7%
Minibatch loss at step 400: 0.773869
Minibatch accuracy: 81.2%
Validation accuracy: 74.5%
Minibatch loss at step 450: 0.960377
Minibatch accuracy: 75.0%
Validation accuracy: 73.4%
Minibatch loss at step 500: 1.182822
Minibatch accuracy: 75.0%
Validation accuracy: 76.0%
Mi

Minibatch loss at step 4600: 0.828757
Minibatch accuracy: 68.8%
Validation accuracy: 79.4%
Minibatch loss at step 4650: 1.213529
Minibatch accuracy: 68.8%
Validation accuracy: 79.6%
Minibatch loss at step 4700: 0.976800
Minibatch accuracy: 75.0%
Validation accuracy: 79.2%
Minibatch loss at step 4750: 1.075863
Minibatch accuracy: 68.8%
Validation accuracy: 78.9%
Minibatch loss at step 4800: 0.342540
Minibatch accuracy: 93.8%
Validation accuracy: 79.4%
Minibatch loss at step 4850: 0.528014
Minibatch accuracy: 93.8%
Validation accuracy: 79.1%
Minibatch loss at step 4900: 1.352603
Minibatch accuracy: 68.8%
Validation accuracy: 78.7%
Minibatch loss at step 4950: 0.467309
Minibatch accuracy: 81.2%
Validation accuracy: 79.1%
Minibatch loss at step 5000: 0.959071
Minibatch accuracy: 68.8%
Validation accuracy: 79.2%
Minibatch loss at step 5050: 0.846731
Minibatch accuracy: 75.0%
Validation accuracy: 79.1%
Minibatch loss at step 5100: 0.806805
Minibatch accuracy: 62.5%
Validation accuracy: 78.8%

Validation accuracy: 80.2%
Minibatch loss at step 9300: 0.196981
Minibatch accuracy: 93.8%
Validation accuracy: 80.6%
Minibatch loss at step 9350: 1.272643
Minibatch accuracy: 68.8%
Validation accuracy: 79.8%
Minibatch loss at step 9400: 0.496037
Minibatch accuracy: 87.5%
Validation accuracy: 80.3%
Minibatch loss at step 9450: 0.647559
Minibatch accuracy: 81.2%
Validation accuracy: 80.6%
Minibatch loss at step 9500: 0.642931
Minibatch accuracy: 75.0%
Validation accuracy: 80.4%
Minibatch loss at step 9550: 1.101918
Minibatch accuracy: 62.5%
Validation accuracy: 80.3%
Minibatch loss at step 9600: 0.525783
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
Minibatch loss at step 9650: 1.010693
Minibatch accuracy: 75.0%
Validation accuracy: 80.2%
Minibatch loss at step 9700: 0.767415
Minibatch accuracy: 62.5%
Validation accuracy: 80.5%
Minibatch loss at step 9750: 0.834522
Minibatch accuracy: 75.0%
Validation accuracy: 80.3%
Minibatch loss at step 9800: 0.705222
Minibatch accuracy: 81.2%